In [151]:
import pandas as pd
import numpy as np

class Node(object):

    def __init__(self, df, col=None, split=None, gini=0.5, depth=0, side=" ", parent=None):
        self.left = None
        self.right = None
        self.parent = parent
        
        self.side = side
        self.col = col
        self.df = df
        self.split = split
        self.gini = gini
        self.depth = depth
        self.c1 = None
        self.c2 = None
        
        self.classification = None
        
    def __repr__(self):
        
        def ds():
            s = ""
            for i in range(0, self.depth):
                s = s + "-"
            return s
        
        if self.classification == 0:
            acc = self.c1 / (self.c1 + self.c2)
        elif self.classification == 1:
            acc = self.c2 / (self.c1 + self.c2)
            
        if self.col is None:
            return "%s%sCount=%d|%d Class=%d Acc=%f" % (ds(), self.side, self.c1, self.c2, self.classification, acc)
        
        return "%s%sCount=%d|%d split(%s)=%s, Gini=%f, Class=%s" % (ds(), self.side, self.c1, self.c2, self.col, self.split, self.gini, self.classification)

    def print(self):
        print(self)
        if self.left is not None:
            self.left.print()
        if self.right is not None:
            self.right.print()        
        
class Tree(object):
    def __init__(self, head):
        self.head = head
        
        buildTree(self.head)
        
    def print(self):
        self.head.print()

        
def bestIntSplit(node, col: str):
    start = node.df[col].min()
    end = node.df[col].max()

    mingini = 0.5
    minsplit = None

    if type(node.df[col].values[0]) == np.int64:
        splits = np.linspace(start, end, 10, dtype=np.int64)
    elif type(node.df[col].values[0]) == np.float64:
        splits = np.linspace(start, end, 10, dtype=np.float64)

    for split in splits:

        split = int(split)

        c1l = node.df[(node.df[col] < split) & (node.df['Donated_in_Mar07'] == 0)].shape[0]
        c2l = node.df[(node.df[col] < split) & (node.df['Donated_in_Mar07'] == 1)].shape[0]

        c1r = node.df[(node.df[col] >= split) & (node.df['Donated_in_Mar07'] == 0)].shape[0]
        c2r = node.df[(node.df[col] >= split) & (node.df['Donated_in_Mar07'] == 1)].shape[0]

        assert(c1l + c2l + c1r + c2r == node.df.shape[0])

        if c1l + c2l == 0 or c1r + c2r == 0:
            continue

        # GINI Index of the left and right branches
        ginil = 1 - ((c1l/(c1l+c2l))**2 + (c2l/(c1l+c2l))**2)
        ginir = 1 - ((c1r/(c1r+c2r))**2 + (c2r/(c1r+c2r))**2)

        # GINI Index of a split
        gini = (c1l + c2l)/(c1l + c2l + c1r + c2r)*ginil + (c1r + c2r)/(c1l + c2l + c1r + c2r)*ginir

        if gini < mingini:
            mingini = gini
            minsplit = split

    return mingini, minsplit


def pure(node):
    
    if node.left is None and node.right is None:
        return True
    
    if node.left.classification == node.right.classification:
        return pure(node.left) and pure(node.right)
    else:
        return False

def prune(node):
    
    if node.parent.left.classification == node.parent.right.classification:
        if pure(node.parent.left) and pure(node.parent.right):
            node.parent.left = None
            node.parent.right = None
            node.parent.col = None
            prune(node.parent)
        else:
            return

def buildTree(node, maxDepth=5, depth=0, usedkeys=[]):
    
    if depth > maxDepth:
        prune(node)
        return
        
    c1 = node.df[node.df['Donated_in_Mar07'] == 0].shape[0]
    c2 = node.df[node.df['Donated_in_Mar07'] == 1].shape[0]
    
    node.c1 = c1
    node.c2 = c2
    
    if c1 > c2:
        node.classification = 0
    else:
        node.classification = 1
        
    mingini = 0.5
    mincol = None
    minsplit = None
    
    for key in node.df.keys():

        if key == 'Donated_in_Mar07':
            continue

        if key in usedkeys:
            continue

        gini, split = bestIntSplit(node, key)

        if gini < mingini:
            mingini = gini
            mincol = key
            minsplit = split
            usedkeys.append(key)
            
    if mincol is None:
        prune(node)
        return

    node.col = mincol
    node.gini = mingini
    node.left = Node(node.df[node.df[mincol] < minsplit], depth=depth+1, side="<", parent=node) 
    node.right = Node(node.df[node.df[mincol] >= minsplit], depth=depth+1, side=">", parent=node)
    node.split = minsplit
    
    buildTree(node.left, depth=depth+1, usedkeys=usedkeys.copy())
    buildTree(node.right, depth=depth+1, usedkeys=usedkeys.copy())
    
    return node

In [152]:
node = Node(pd.read_csv('data.csv'))
tree = Tree(node)
tree.print()

 Count=570|178 split(Recency_in_months)=8, Gini=0.328680, Class=0
-<Count=237|138 split(Frequency_times)=6, Gini=0.445563, Class=0
--<Count=153|60 split(Monetary_blood_vol)=1027, Gini=0.396197, Class=0
---<Count=133|45 Class=0 Acc=0.747191
--->Count=20|15 split(Months_Since_1st_Donation)=18, Gini=0.350649, Class=0
----<Count=2|9 Class=1 Acc=0.818182
---->Count=18|6 Class=0 Acc=0.750000
-->Count=84|78 split(Months_Since_1st_Donation)=42, Gini=0.456074, Class=0
---<Count=26|47 Class=1 Acc=0.643836
--->Count=58|31 Class=0 Acc=0.651685
->Count=333|40 Class=0 Acc=0.892761
